### Import ES & Libs

In [1]:
import pickle
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

In [7]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [18]:
# get exist data
res = es.search(index="people", body={"query": {"match_all": {}}})
print(res)

{'timed_out': False, '_shards': {'failed': 0, 'total': 3, 'successful': 3, 'skipped': 0}, 'took': 90, 'hits': {'max_score': 1.0, 'total': 7, 'hits': [{'_source': {'age': '35', 'country': 'China', 'name': '大昌意', 'date': '1982-10-14'}, '_score': 1.0, '_index': 'people', '_type': 'man', '_id': 'AV-UVGo0dGVqd-b-kBV6'}, {'_source': {'timed': '14', 'timestamp': '[22/Jan/2018:15:02:01', 'action': 'GET /css/index.css HTTP/1.1', 'status': '200', 'respsize': '945', 'data': 'director 123456 136.133.92.208 1 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}, '_score': 1.0, '_index': 'people', '_type': 'log', '_id': '0'}, {'_source': {'timed': '2', 'timestamp': '[22/Jan/2018:15:02:03', 'action': 'GET / HTTP/1.1', 'status': '200', 'respsize': '1040', 'data': 'administrator 123456 30.142.97.121 1 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}, '_score': 

### Index data

In [24]:
# import file
BF_logs_raw = pd.read_csv("./data/BF_datalab_httpd_all.log", header = None, delimiter = " ")
log_length = BF_logs_raw.shape[0]
BF_logs_raw.head(1)

,0,1,2,3,4,5,6,7,8,9,10
0,::ffff:46.20.170.141,-,-,[22/Jan/2018:15:02:01,+0000],GET / HTTP/1.1,200,1040,-,director 123456 136.133.92.208 1 Mozilla/5.0 (...,312


In [27]:
from elasticsearch import helpers

def bulkData(length, log_type):
    es = Elasticsearch()
    j = 0
    actions = []
    while (j < length):
        log = BF_logs_raw.loc[j]
        action = {
            "_index": 'log',
            "_type": log_type,
            "_id": j,
            "_source": {
                "timestamp":log[3],
                "action":log[5],
                "status":log[6],
                "respsize":log[7],
                "data":log[9],
                "timed":log[10]
            }
        }
        actions.append(action)
        j += 1
    # use bulk to POST data
    if (len(actions) > 0):
        helpers.bulk(es, actions)
        del actions[0:len(actions)]

In [28]:
# execute function to store data
bulkData(log_length, 'bf')

#### For other logs

In [32]:
normal_logs_raw = pd.read_csv("./data/normal.log", header = None, delimiter = " ")
sql_logs_raw = pd.read_csv("./data/sql.log", header = None, delimiter = " ")
xss_logs_raw = pd.read_csv("./data/xss.log", header = None, delimiter = " ",error_bad_lines=False)

log_length_normal = normal_logs_raw.shape[0]
log_length_sql = sql_logs_raw.shape[0]
log_length_xss = xss_logs_raw.shape[0]

b'Skipping line 79: expected 24 fields, saw 26\nSkipping line 80: expected 24 fields, saw 26\nSkipping line 81: expected 24 fields, saw 26\nSkipping line 82: expected 24 fields, saw 26\nSkipping line 83: expected 24 fields, saw 26\nSkipping line 84: expected 24 fields, saw 25\nSkipping line 85: expected 24 fields, saw 25\nSkipping line 86: expected 24 fields, saw 25\nSkipping line 87: expected 24 fields, saw 25\nSkipping line 88: expected 24 fields, saw 25\nSkipping line 89: expected 24 fields, saw 25\nSkipping line 180: expected 24 fields, saw 27\nSkipping line 181: expected 24 fields, saw 27\nSkipping line 182: expected 24 fields, saw 27\nSkipping line 183: expected 24 fields, saw 27\nSkipping line 184: expected 24 fields, saw 27\nSkipping line 248: expected 24 fields, saw 25\nSkipping line 249: expected 24 fields, saw 25\nSkipping line 250: expected 24 fields, saw 25\nSkipping line 251: expected 24 fields, saw 25\nSkipping line 252: expected 24 fields, saw 25\nSkipping line 299: exp

In [33]:
def bulkData(length, log_type):
    es = Elasticsearch()
    j = 0
    actions = []
    # give diffrent separators to diffrent logs
    if log_type == 'bf':
        separator = 0
    if log_type == 'normal':
        separator = 200000
    if log_type == 'sql':
        separator = 400000
    if log_type == 'xss':
        separator = 600000
    while (j < length):
        log = BF_logs_raw.loc[j]
        action = {
            "_index": 'log',
            "_type": log_type,
            "_id": j + separator,
            "_source": {
                "timestamp":log[3],
                "action":log[5],
                "status":log[6],
                "respsize":log[7],
                "data":log[9],
                "timed":log[10]
            }
        }
        actions.append(action)
        j += 1
    # use bulk to POST data
    if (len(actions) > 0):
        helpers.bulk(es, actions)
        del actions[0:len(actions)]

In [34]:
bulkData(log_length_normal, 'normal')
bulkData(log_length_sql, 'sql')
bulkData(log_length_xss, 'xss')

### Get data 

In [56]:
es.search(index="log", doc_type='normal',body={"query": {"match_all": {}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '200000',
    '_index': 'log',
    '_score': 1.0,
    '_source': {'action': 'GET / HTTP/1.1',
     'data': 'director 123456 136.133.92.208 1 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36',
     'respsize': '1040',
     'status': '200',
     'timed': '312',
     'timestamp': '[22/Jan/2018:15:02:01'},
    '_type': 'normal'},
   {'_id': '200001',
    '_index': 'log',
    '_score': 1.0,
    '_source': {'action': 'GET /css/index.css HTTP/1.1',
     'data': 'director 123456 136.133.92.208 1 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36',
     'respsize': '945',
     'status': '200',
     'timed': '14',
     'timestamp': '[22/Jan/2018:15:02:01'},
    '_type': 'normal'},
   {'_id': '200015',
    '_index': 'log',
    '_score': 1.0,
    '_source': {'act